In [25]:
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import io, color, transform

import cv2
import numpy as np
import os
import pandas as pd
import re

from PIL import Image
from sklearn.preprocessing import MinMaxScaler

# READ DATA

In [26]:
#data_train = pd.read_csv("/Users/lethanhtien/Desktop/run/data/train.csv")
data = pd.read_csv("dataset.csv")

# CHECK CLASS IMBALANCE

In [27]:
class_counts = data['label'].value_counts()
print(class_counts)

label
1    1000
2    1000
3    1000
4    1000
0     999
Name: count, dtype: int64


In [28]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ cân bằng lớp
class_counts.plot(kind='bar', title='Class Distribution')
plt.show()


<Figure size 640x480 with 1 Axes>

In [29]:
total_samples = len(data)
class_ratios = class_counts / total_samples
print(class_ratios)


label
1    0.20004
2    0.20004
3    0.20004
4    0.20004
0    0.19984
Name: count, dtype: float64


# SPLIT DATA

In [30]:
# x_train = data_train.drop("label", axis=1)
# y_train = data_train['label']

# x_test = data_test.drop("label", axis=1)
# y_test = data_test['label']

x = data.drop("label", axis=1)
y = data['label']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

In [32]:
X_train[0]

array([ 28,  23,  24, ..., 166, 211, 218], dtype=int64)

# SCALE DATA

In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
# Khởi tạo mô hình MinMaxScaler và fit trên tập huấn luyện
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Sử dụng mô hình đã fit để chuẩn hóa tập kiểm tra
X_test_scaled = scaler.transform(X_test)

# TRAINING

In [36]:
from sklearn.svm import SVC
# Huấn luyện mô hình SVM với các tham số cụ thể
svm_model = SVC(C=0.1, gamma=0.1, kernel='linear', probability=True)
svm_model.fit(X_train_scaled, y_train)

# Dự đoán nhãn trên tập kiểm tra
y_pred = svm_model.predict(X_test_scaled)

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.616


# EVALUATE

In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Tính precision, recall, và f1-score cho từng lớp
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

class_names = ['Angry', 'Fear', 'Happy', 'Neutral', 'Sad']

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}\n')

for i in range(len(class_names)):
    print(f"Class {class_names[i]}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall: {recall[i]:.4f}")
    print(f"  F1-Score: {f1[i]:.4f}")
    print()


Accuracy: 0.616

Class Angry:
  Precision: 0.5337
  Recall: 0.5236
  F1-Score: 0.5286

Class Fear:
  Precision: 0.4947
  Recall: 0.4677
  F1-Score: 0.4808

Class Happy:
  Precision: 0.8376
  Recall: 0.8128
  F1-Score: 0.8250

Class Neutral:
  Precision: 0.7835
  Recall: 0.8085
  F1-Score: 0.7958

Class Sad:
  Precision: 0.4455
  Recall: 0.4796
  F1-Score: 0.4619



# TEST

In [39]:
def test(path):
    # Đường dẫn của tấm ảnh
    image_path = path

    # Mở ảnh bằng Pillow
    img = Image.open(image_path)

    # Chuyển ảnh về ảnh trắng đen
    img = img.convert("L")

    # Chuyển ảnh thành mảng NumPy
    img_array = np.array(img)

    # Resize ảnh về kích thước mong muốn
    resized_img = cv2.resize(img_array, (48, 48))

    flattened_img = resized_img.flatten().reshape(-1, 1)
    # Use MinMaxScaler
    scaler = MinMaxScaler()
    img1 = scaler.fit_transform(flattened_img).T

    y_pred = svm_model.predict(img1)

    return y_pred

test('images/angry/image0000390.jpg')

array([0], dtype=int64)

# SAVE MODEL

In [40]:
from joblib import dump
dump(svm_model, 'svm_model.joblib')

['svm_model.joblib']

# TRAINING

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Thiết lập các giá trị thử nghiệm cho C, kernel và gamma
param_grid = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf'],
              'gamma': [0.1, 1, 10]}

# Tạo mô hình SVM
svm_model = SVC()

# Sử dụng GridSearchCV để tìm kiếm qua lưới tham số
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best Parameters:", grid_search.best_params_)

# Đánh giá mô hình trên tập kiểm tra
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Best Parameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Accuracy: 0.579


# GRADIO.APP INTERFACE


In [ ]:
%pip install --upgrade gradio


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



                                              0.0/16.6 MB ? eta -:--:--
                                              0.0/16.6 MB ? eta -:--:--
                                             0.0/16.6 MB 330.3 kB/s eta 0:00:51
                                             0.0/16.6 MB 330.3 kB/s eta 0:00:51
                                             0.1/16.6 MB 365.7 kB/s eta 0:00:46
                                             0.1/16.6 MB 602.4 kB/s eta 0:00:28
                                              0.3/16.6 MB 1.4 MB/s eta 0:00:12
     -                                        0.4/16.6 MB 1.7 MB/s eta 0:00:10
     --                                       0.9/16.6 MB 2.9 MB/s eta 0:00:06
     ---                                      1.3/16.6 MB 3.9 MB/s eta 0:00:04
     ----                                     1.8/16.6 MB 5.1 MB/s eta 0:00:03
     -----                                    2.3/16.6 MB 5.7 MB/s eta 0:00:03
     ------                                   2.8/16.6 MB 6.6

In [41]:
import gradio as gr
import numpy as np
from PIL import Image
from joblib import load
import dlib

svm_model = load('svm_model.h5')

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Happy",
    3: "Neutral",
    4: "Sad",
}

# Load dlib's pre-trained face detector
detector = dlib.get_frontal_face_detector()

def predict_emotion(image):
    # Convert Gradio Image to NumPy array
    img_array = np.array(image)

    # Convert to grayscale
    processed_image = Image.fromarray(img_array).convert('L').resize((48, 48))

    # Convert to NumPy array
    processed_image_array = np.array(processed_image)

    # Flatten and reshape
    flattened_img = processed_image_array.flatten().reshape(1, -1) / 255.0

    # Make prediction using SVM model
    prediction = svm_model.predict(flattened_img)[0]

    # Detect faces using dlib
    faces = detector(img_array, 1)
    
    if len(faces) == 0:
        cropped_face = img_array
    else:
        # Assuming there's only one face detected, you can modify this if needed
        x, y, w, h = faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()
        cropped_face = img_array[y:y+h, x:x+w]

    print("Input image shape:", img_array.shape)
    print("Processed image shape:", processed_image_array.shape)

    return emotion_labels[prediction], cropped_face


iface = gr.Interface(
    fn=predict_emotion,
    inputs="image",
    outputs=["text", "image"],  # Specify both text and image outputs
    live = True,
    title="Emotion Detection",
    description="Upload an image and predict the emotion."
)

iface.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Input image shape: (704, 800, 3)
Processed image shape: (48, 48)
